In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip install tensorflow numpy tqdm pandas

**Creating DataFrames**

In [4]:
goodreads_train = pd.read_csv('../kaggle/input/goodreads_train.csv')
goodreads_test = pd.read_csv('../kaggle/input/goodreads_test.csv')

In [5]:
goodreads_train.shape

(900000, 11)

In [6]:
goodreads_test.shape

(478033, 10)

In [7]:
goodreads_train.sample(5)

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
50896,88210a9e22b08c0f025e0e962a177ba3,17825130,462d0015f95dbfca6111ab68133592f9,4,TRIGGER WARNING: Hate-crime. \n So your parent...,Thu May 22 21:59:13 -0700 2014,Sat Mar 28 05:46:23 -0700 2015,Sat Mar 28 07:11:38 -0700 2015,NaN,2,0
797757,c6097973f4c03f4039d0f57a539650da,13092528,bcdc7748588583a366f7c5580dacec60,3,This series started out with some promise even...,Fri Feb 03 23:18:37 -0800 2017,Fri Feb 03 23:25:29 -0800 2017,Fri Feb 03 00:00:00 -0800 2017,NaN,0,0
334860,9db1fd33657796dfee932f86f7b43c2e,29906980,a3e4b8d2c1e404e12a6460a69ed17729,2,2.5 stars. \n Huh. \n That ... was one of the ...,Mon Mar 20 14:54:24 -0700 2017,Sat May 20 23:51:58 -0700 2017,Sat May 20 23:47:27 -0700 2017,Sat May 20 23:47:22 -0700 2017,0,0
793053,56be7da8cf301ae22f8d2ea53a528fe4,12936,ab5e10d97dd3d5474e9584d430685559,3,"First off, I *loved* The Giver. I gave it 5 st...",Tue Dec 20 22:09:33 -0800 2011,Mon May 02 22:31:30 -0700 2016,Tue Jan 24 21:49:36 -0800 2012,Wed Dec 14 00:00:00 -0800 2011,0,3
261968,3af5f535463d79601c920e556286389b,23312692,768934521b49685d858ad023daa20841,5,I spent a good portion of this book KNOWING th...,Sun May 10 05:37:10 -0700 2015,Tue May 12 06:01:40 -0700 2015,NaN,Sun May 10 00:00:00 -0700 2015,0,0


**Cleaning DataFrames**

In [8]:
goodreads_train['rating'].value_counts()

4    313688
5    265007
3    188972
2     72627
0     30988
1     28718
Name: rating, dtype: int64

In [9]:
goodreads_train.set_index("review_id", inplace=True)
goodreads_test.set_index("review_id", inplace=True)

In [10]:
books = pd.concat([goodreads_train.drop('rating', axis=1), goodreads_test], axis=0)
books_rates = goodreads_train['rating']

In [11]:
books.shape

(1378033, 9)

In [12]:
books.sample(5)

,user_id,book_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
review_id,,,,,,,,,
280ec1fa4418ca5365a1d69a76d67b7e,708c6f1819d0ffebf9b10f4870053fca,22295304,4.5 stars \n A book about people that can take...,Fri Apr 07 10:20:29 -0700 2017,Wed Apr 12 09:11:32 -0700 2017,Wed Apr 12 09:11:33 -0700 2017,Fri Apr 07 00:00:00 -0700 2017,0,0
e9a9002c61b8fb4dd503dfaf507dee94,821c1b4a63183de4359ee673748841eb,7230431,"Another very good novel from Cherise Sinclair,...",Fri Jan 20 05:33:31 -0800 2012,Wed Apr 10 15:28:04 -0700 2013,Wed Apr 10 15:28:04 -0700 2013,Mon Apr 01 00:00:00 -0700 2013,0,0
e55add294854d15e6da41526de219d03,d9af8189dd318aabf18f475e4051df8a,17158513,That ending... \n I can't even...I cant... My ...,Thu Aug 01 10:40:31 -0700 2013,Thu Jan 09 17:06:05 -0800 2014,Thu Aug 01 00:00:00 -0700 2013,Wed Jul 31 00:00:00 -0700 2013,0,0
acb8f2e30902167e661350e17349ce5e,9da810ac28645a3d958c525735d92747,107779,I so enjoyed this book!,Mon Oct 07 19:45:21 -0700 2013,Fri Oct 18 00:43:57 -0700 2013,NaN,NaN,0,2
91d7b44a625d21ad4b0fb304a8cbb17d,57af73411c19a4493ef9bfd6f02b8f7f,17730557,"Good, but seems like more of a beginning than ...",Wed Jun 05 11:45:44 -0700 2013,Fri Sep 06 21:21:29 -0700 2013,Wed Jul 24 00:00:00 -0700 2013,NaN,1,0


In [13]:
books.drop(columns=['date_updated', 'read_at', 'started_at'], axis=1, inplace=True)

In [14]:
# Remove negative notes
books = books[(books['n_votes'] >= 0) & (books['n_comments'] >= 0)]

In [15]:
# Checking duplicates
sum(books.duplicated())

0

In [16]:
sum(books.duplicated(subset=['review_text']))

16130

In [17]:
books.drop_duplicates(subset=['review_text'], inplace=True)

In [18]:
unapp_texts = books['review_text'].apply(lambda txt: np.mean(list(map(len, txt.split())))) > 25
unapp_texts.shape

(1361800,)

In [19]:
books[unapp_texts].head()

,user_id,book_id,review_text,date_added,n_votes,n_comments
review_id,,,,,,
ba7e149d70cb858d87d4113dfe0e4091,8092e33e9c50d3e269a8164177d2abdc,6339664,IhavetoadmitihavefalleninlovewithPatcicannotwa...,Sun Sep 30 12:29:48 -0700 2012,1,1
90529a603506931b91502b3c65b0564c,308c545182f8b5d332ff71c667fd9b61,12127810,the feels for NICO DI ANGELO.....................,Thu May 16 23:11:00 -0700 2013,0,0
7105506d7aa373be892e6d7d29750741,302eb3debfc02e0bdd0fd617d34a2713,10644930,"""The past is obdurate."" \n rtcrtcrtcrtcrtcrtcr...",Fri Mar 18 07:59:13 -0700 2016,0,0
ba4e577524730630bef88cb4c76c6dd7,de107a99cb0ed2ec5d90d744cc48c310,23723788,2.5 it's-a-good-book-as-in-even-after-months-I...,Sun Aug 30 18:18:44 -0700 2015,0,0
d9d104943954cc9d975a07d1a724be90,de107a99cb0ed2ec5d90d744cc48c310,23496782,2.5 It-Was-a-Very-Okay-Book-With-the-Mystery-a...,Thu Apr 09 04:41:42 -0700 2015,3,0


In [20]:
books.drop(books[unapp_texts].index, axis=0, inplace=True)

In [21]:
spoil_statement = 'spoiler alert'
books['spoil'] = [True if spoil_statement in txt else False for txt in books['review_text']]

In [22]:
books[books['spoil'] == True]

,user_id,book_id,review_text,date_added,n_votes,n_comments,spoil
review_id,,,,,,,
e23965adda7ce7e7e415a1339e169065,8842281e1d1347389f2ab93d60773d4d,62291,** spoiler alert ** \n Loved it. The epic saga...,Tue Jul 12 12:57:07 -0700 2011,5,1,True
0258b3c9a85cecb95f240e43a7642f60,8842281e1d1347389f2ab93d60773d4d,7112495,** spoiler alert ** \n An interesting book in ...,Thu Jun 10 14:41:13 -0700 2010,7,3,True
d44b6d28f456dbcc5b1537b4c5572400,8842281e1d1347389f2ab93d60773d4d,6411961,"** spoiler alert ** \n Critics aside, Dan Brow...",Wed Sep 16 11:09:03 -0700 2009,3,3,True
d49eb786ef1d8557b9e6e86bcafd4570,8842281e1d1347389f2ab93d60773d4d,77432,"** spoiler alert ** \n I equally loved book 2,...",Tue May 05 18:08:22 -0700 2009,1,0,True
b8e08587ce1592d69548d0f644cb9893,8842281e1d1347389f2ab93d60773d4d,14497,** spoiler alert ** \n Just finished this and ...,Thu Mar 27 17:53:06 -0700 2008,6,1,True
...,...,...,...,...,...,...,...
a989c818917a7463089454b356fb6671,5ff0927d425ab1a70d2b2d41fe6b96b5,13239950,** spoiler alert ** \n Hills like White Elepha...,Wed Oct 31 18:22:50 -0700 2012,25,7,True
3736d5ee5904b4b5651912dd28b7ce01,a16fd6b48c6e877042acbe2bda8a0007,115076,** spoiler alert ** \n I was completely engros...,Mon Jun 04 07:18:34 -0700 2012,0,0,True
a7777a1c18ab582586a61ad453d91949,692ce8eb4d5490980f6e74f6cb8671e2,12974372,** spoiler alert ** \n Hmm...what to say about...,Wed Nov 04 12:33:22 -0800 2015,1,0,True
